# Ribociclib CCLE GDSC2

This notebook investigates the effect of ribociclib on cell lines from the Cancer Cell Line Encyclopedia (CCLE) and the Genomics of Drug Sensitivity in Cancer (GDSC2) datasets.

## Run 1 - Naive Feature Selection 

In [9]:
## LOADING DATA

import pickle

# Load ribociclib data from data/preprocessed 

with open('data/preprocessed/ribociclib_data.pkl', 'rb') as f:
    ribociclib_data = pickle.load(f)

In [10]:
import dynmarker.FeatureSelection as fs 

In [11]:
feature_data, label_data = fs.create_feature_and_label(ribociclib_data)

In [12]:
feature_data.head()

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,H3C2,H3C3,AC098582.1,DUS4L-BCAP29,C8orf44-SGK3,ELOA3B,NPBWR1,ELOA3D,ELOA3,CDR1
SANGER_MODEL_ID,,,,,,,,,,,,,,,,,,,,,
SIDM00872,5.296090,0.000000,6.794416,3.452859,5.260778,0.042644,4.339137,5.833143,7.313246,4.764474,...,1.827819,0.000000,0.847997,1.704872,0.411426,0.000000,1.063503,0.000000,0.000000,0.000000
SIDM00866,5.214125,0.000000,6.328047,3.168321,4.654206,0.042644,0.432959,6.574102,5.140779,5.114783,...,0.000000,0.000000,0.298658,1.584963,0.695994,0.000000,2.693766,0.000000,0.000000,0.000000
SIDM00885,5.241840,0.201634,5.615299,3.090853,3.732269,0.084064,0.111031,4.026800,5.471513,6.019480,...,0.632268,0.000000,0.422233,1.250962,0.505891,0.111031,0.070389,0.000000,0.111031,0.000000
SIDM00884,3.481557,0.000000,7.070389,2.341986,3.679199,0.176323,3.420887,5.854494,5.976134,5.112283,...,2.327687,0.226509,0.650765,2.477677,0.000000,0.000000,0.042644,0.000000,0.084064,0.000000
SIDM00877,4.349082,0.000000,6.479942,2.790772,4.031219,0.070389,0.070389,4.682011,5.632850,4.441616,...,0.400538,0.084064,0.150560,1.823749,0.163499,0.042644,0.097611,0.084064,0.042644,0.475085


In [13]:
label_data.head()

SANGER_MODEL_ID
SIDM00872    4.254618
SIDM00866    3.583018
SIDM00885    4.023289
SIDM00884    3.146215
SIDM00877    6.135124
Name: LN_IC50, dtype: float64

In [14]:
feature_data.shape

(44, 19221)

In [15]:
label_data.shape

(44,)

In [16]:
naive_scores = fs.naive_test_regression(feature_data, label_data, cv=5, verbose=1)

------ Naive test - Linear Regression
Linear Regression score: 1.0558, std: 0.3175


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score

scaler = Normalizer()
models = [LinearRegression(), Ridge(), Lasso(), ElasticNet(), LinearSVR(max_iter=100000), KNeighborsRegressor(), RandomForestRegressor(), MLPRegressor(hidden_layer_sizes=(128, 64, 32), max_iter=20000, alpha=1)]

for model in models:
    pipe = Pipeline([('scaler', scaler), ('regressor', model)])
    scores = cross_val_score(pipe, mrmr_feature_selected_data, label_data, cv=5, scoring='neg_mean_squared_error')
    print(f'------------ {model.__class__.__name__}')
    print(f'NORM mean: {-scores.mean():.4f}, std: {scores.std():.4f}')

    scores = cross_val_score(model, mrmr_feature_selected_data, label_data, cv=5, scoring='neg_mean_squared_error')
    print(f'NONE mean: {-scores.mean():.4f}, std: {scores.std():.4f}')

In [ ]:
# import f_regression from sklearn.feature_selection
from sklearn.feature_selection import f_regression

selected_feature_scores, support = fs.filter_feature_selection(feature_data, label_data, method=f_regression, K=100)

(44, 100)


In [ ]:
support

array([   25,    59,   307,   492,  1118,  1273,  1368,  1469,  1789,
        1840,  1926,  2650,  2806,  2862,  2868,  3720,  4275,  4557,
        4817,  5339,  5463,  5642,  5827,  6043,  6183,  6290,  6561,
        6625,  6629,  7350,  8108,  8182,  8424,  8593,  8850,  8949,
        9171,  9735,  9767, 10040, 10070, 10439, 10604, 10627, 10752,
       11086, 11092, 11230, 11308, 11436, 11563, 12068, 12147, 12361,
       12449, 12538, 12541, 12543, 12546, 12584, 12610, 12623, 12650,
       12753, 12833, 12917, 13001, 13696, 13848, 13882, 13971, 14604,
       14748, 15190, 15484, 15615, 15709, 15758, 15788, 16305, 16653,
       16674, 16720, 16745, 16792, 16881, 17726, 17773, 17831, 17987,
       18139, 18241, 18269, 18443, 18564, 18835, 18955, 19096, 19146,
       19198], dtype=int64)

In [ ]:
selected_feature_scores

array([10.82627678, 15.92844663, 13.71697482,  9.95307355, 12.13242575,
       10.02079788, 19.75970755, 11.90586857, 11.22664203, 12.33704868,
        9.48603855,  9.8415579 , 10.81133646,  9.37640248,  9.6787931 ,
       24.05328882,  9.2083788 , 11.9992517 ,  9.78245435,  9.59244357,
        9.715793  , 11.30495425, 11.00389816,  9.71452309, 22.02057141,
       10.677195  , 10.03519764, 12.70750482, 11.35548111,  9.64648155,
       12.34158029, 23.2071735 ,  9.75107191,  9.45111478,  9.60400579,
       12.41447586,  9.31145859,  9.5312938 ,  9.76527142, 10.89872446,
        9.64590581,  9.20248742, 13.75909631, 13.24306591, 14.72834766,
        9.93772295, 14.84494037, 15.14213456, 14.11271046, 10.93071752,
       14.98189937, 10.50886893, 11.23182619, 11.72389423,  9.75146395,
        9.49045437,  9.23285322, 13.58738927,  9.48121941,  9.28324863,
        9.31968651, 10.02964415,  9.21877019, 10.40267397,  9.54049067,
        9.97512081, 11.32237901,  9.43377427, 16.91962829, 13.16

In [ ]:
selected_feature_data = feature_data.iloc[:, support]
selected_feature_data.head()

,WNT16,PDK4,GABRA3,ALG1,CHAT,TXK,PAK3,MOXD1,TESC,ARHGAP4,SLC17A6,BEX4,NOMO1,ZDHHC2,FZD3,FOLR3,HEMK1,MFN2,FGF23,PLP1,PSG8,PAX1,RAB3IP,MPDU1,CACNG6,AKAP12,ZSWIM3,BEX2,BEX1,VARS2,CACNG8,SYPL2,ZNF385B,SHISAL2B,GOLGA7,IDI2,SPATA4,OTOA,MICU3,GPRASP2,...,SLC26A5,OR10A3,PSG6,NPTX1,CLSTN1,FOXB1,CSDC2,ZNF135,OR2T33,OR2T8,DPY19L2,NLRP10,SLC8A1,NOMO2,RGS7BP,VSTM2B,SLC18A3,CLEC17A,TMEM221,ZFP28,ZNF334,PEG3,CNGA1,ZNF536,LRRTM3,BHLHB9,IFI30,PSG7,C1GALT1C1L,PSG1,OR2W3,PSG2,PSG11,ZNF345,MTRNR2L10,ZIM2,TBC1D3I,OR10A6,CSNKA2IP,AC113348.1
SANGER_MODEL_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SIDM00872,0.275007,0.333424,1.510962,4.617063,0.014355,1.000000,0.028569,0.226509,0.704872,2.039138,0.042644,3.148934,6.175325,0.475085,1.963474,0.070389,3.795975,6.910133,0.028569,0.124328,0.028569,0.000000,3.894333,6.197512,0.097611,1.286881,1.448901,2.853996,1.097611,4.313246,0.084064,0.028569,0.604071,0.0,6.804647,0.000000,0.084064,0.000000,0.056584,2.378512,...,0.014355,0.042644,0.070389,0.238787,6.876885,0.014355,0.014355,0.042644,0.000000,0.000000,0.084064,0.000000,0.594549,6.967745,0.150560,0.028569,0.000000,0.028569,0.084064,0.687061,0.056584,0.739848,2.295723,0.014355,0.0,0.464668,7.368856,0.0,0.000000,0.275007,0.000000,0.000000,0.084064,2.032101,0.0,0.070389,0.238787,0.084064,0.0,0.000000
SIDM00866,0.411426,0.238787,0.028569,5.426265,0.000000,1.292782,0.042644,0.748461,0.176323,2.901108,0.014355,1.201634,6.420550,2.680324,1.275007,0.000000,3.144046,5.848247,0.000000,0.201634,0.400538,0.000000,5.260026,6.120186,0.028569,2.253989,2.114367,2.435629,0.176323,4.948601,0.000000,0.014355,0.238787,0.0,5.185470,0.000000,0.056584,0.042644,1.035624,1.356144,...,0.000000,0.028569,0.070389,0.176323,7.032542,0.070389,0.201634,1.144046,0.000000,0.000000,1.280956,0.948601,0.042644,7.572890,0.333424,0.000000,0.000000,0.014355,0.443607,1.490570,0.014355,0.028569,0.176323,0.014355,0.0,0.985500,7.917551,0.0,0.028569,0.137504,0.000000,0.000000,0.000000,1.130931,0.0,0.000000,0.454176,0.014355,0.0,0.000000
SIDM00885,0.056584,2.424922,0.150560,3.653060,0.042644,0.124328,0.298658,0.863938,0.263034,2.636915,0.000000,4.539779,5.755422,0.124328,2.313246,0.176323,2.526069,6.073820,0.000000,0.214125,0.000000,0.000000,4.421560,5.165108,0.014355,2.063503,2.014355,4.648465,1.189034,4.775577,0.000000,0.028569,0.495695,0.0,5.453518,0.028569,0.028569,0.056584,0.765535,2.763412,...,0.084064,0.014355,0.000000,0.124328,5.261907,0.028569,0.613532,3.618239,0.000000,0.000000,0.641546,0.298658,0.176323,6.456970,0.263034,0.000000,0.014355,0.000000,0.028569,2.666757,1.636915,0.214125,0.226509,0.000000,0.0,2.042644,6.684959,0.0,0.454176,0.263034,0.000000,0.000000,0.000000,2.875780,0.0,0.000000,0.622930,0.014355,0.0,0.137504
SIDM00884,0.124328,0.070389,0.028569,4.750070,0.014355,0.028569,0.028569,1.056584,0.214125,1.608809,0.000000,1.769772,7.599392,2.594549,0.948601,0.250962,3.960697,6.747924,0.000000,0.137504,0.000000,0.000000,1.232661,6.006747,0.014355,0.887525,2.097611,3.878725,0.731183,4.878235,0.084064,0.111031,0.097611,0.0,4.728465,0.000000,0.000000,0.014355,0.970854,0.985500,...,0.000000,0.000000,0.000000,1.014355,6.410070,0.000000,0.475085,2.198494,0.000000,0.000000,0.226509,0.111031,1.028569,8.238214,0.000000,0.000000,0.014355,0.000000,0.189034,1.739848,0.111031,3.249445,0.056584,0.000000,0.0,0.097611,7.061560,0.0,0.422233,0.137504,0.124328,0.042644,0.000000,1.536053,0.0,0.400538,0.084064,0.000000,0.0,0.000000
SIDM00877,0.070389,0.111031,0.000000,2.702658,0.000000,0.475085,0.555816,0.516015,0.201634,2.347666,0.000000,2.829850,4.338424,2.432959,1.974529,0.432959,3.109361,5.269407,0.014355,0.150560,0.000000,0.389567,5.313609,3.806324,0.014355,0.687061,0.739848,6.407353,0.933573,3.925050,0.070389,0.632268,0.378512,0.0,6.438459,0.028569,0.014355,0.028569,1.570463,1.438293,...,0.028569,0.000000,0.000000,0.097611,3.725741,0.056584,0.238787,1.981853,0.275007,0.124328,2.397803,0.000000,1.485427,4.815575,0.000000,0.056584,0.000000,0.014355,0.042644,2.

In [ ]:
f_regression_filter_scores = fs.naive_test_regression(selected_feature_data, label_data, cv=5)

In [ ]:
### MRMR Feature Selection

from dynmarker.FeatureSelection import mrmr_select_fcq

mrmr_feature_selected, successive_scores = mrmr_select_fcq(feature_data, label_data, K=50, verbose=1)

Iteration 1 selected FOLR3 score 2405328.882360086 remaining 19220 features
Iteration 2 selected KIAA1841 score 56175.64972741656 remaining 19219 features
Iteration 3 selected COX6A2 score 929.9799119433212 remaining 19218 features
Iteration 4 selected PGA5 score 243.06331156021403 remaining 19217 features
Iteration 5 selected L3MBTL2 score 245.8932639037736 remaining 19216 features
Iteration 6 selected PAK3 score 150.1157165100612 remaining 19215 features
Iteration 7 selected OR2T33 score 101.45558493617858 remaining 19214 features
Iteration 8 selected SYPL2 score 85.52732482067111 remaining 19213 features
Iteration 9 selected CACNG6 score 73.9764396367321 remaining 19212 features
Iteration 10 selected TBC1D3I score 74.62674289019189 remaining 19211 features
Iteration 11 selected SLC8A1 score 63.80905537603905 remaining 19210 features
Iteration 12 selected UVRAG score 63.31609411855542 remaining 19209 features
Iteration 13 selected PDK4 score 62.796959241539774 remaining 19208 feature

In [ ]:
mrmr_feature_selected_data = feature_data.iloc[:, mrmr_feature_selected]
mrmr_feature_selected_data.head()

,FOLR3,KIAA1841,COX6A2,PGA5,L3MBTL2,PAK3,OR2T33,SYPL2,CACNG6,TBC1D3I,SLC8A1,UVRAG,PDK4,TMED10,CSNKA2IP,LRRTM3,CNGA1,VSTM2B,RNF180,CLEC17A,B3GNT2,C8orf48,NOMO1,GABRA3,NTNG1,PAX1,BHLHB9,BEX2,MTRNR2L10,MFN2,NOS1AP,TRIM58,ZSWIM3,TESC,AVPR1A,PPP1R9A,MPDU1,HEMK1,ZIM2,GYG2,SMCP,ALG5,ARHGAP4,TSPAN6,AC113348.1,FGF23,CACNG8,MOXD1,PRKACG,HCRT
SANGER_MODEL_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SIDM00872,0.070389,3.447579,0.000000,0.000000,3.879706,0.028569,0.000000,0.028569,0.097611,0.238787,0.594549,3.821710,0.333424,8.458735,0.0,0.0,2.295723,0.028569,0.028569,0.028569,4.531069,0.000000,6.175325,1.510962,0.070389,0.000000,0.464668,2.853996,0.0,6.910133,0.941106,0.042644,1.448901,0.704872,0.000000,1.157044,6.197512,3.795975,0.070389,4.017922,0.0,5.198494,2.039138,5.296090,0.000000,0.028569,0.084064,0.226509,0.000000,0.0
SIDM00866,0.000000,3.692650,0.000000,0.000000,5.111031,0.042644,0.000000,0.014355,0.028569,0.454176,0.042644,3.845992,0.238787,7.915222,0.0,0.0,0.176323,0.000000,0.545968,0.014355,4.455492,1.150560,6.420550,0.028569,0.137504,0.000000,0.985500,2.435629,0.0,5.848247,1.713696,0.000000,2.114367,0.176323,0.000000,1.604071,6.120186,3.144046,0.000000,0.028569,0.0,4.773469,2.901108,5.214125,0.000000,0.000000,0.000000,0.748461,0.000000,0.0
SIDM00885,0.176323,3.430285,0.000000,0.070389,3.785551,0.298658,0.000000,0.028569,0.014355,0.622930,0.176323,3.397803,2.424922,7.366147,0.0,0.0,0.226509,0.000000,1.627607,0.000000,3.552131,0.000000,5.755422,0.150560,0.042644,0.000000,2.042644,4.648465,0.0,6.073820,2.575312,0.028569,2.014355,0.263034,0.056584,2.140779,5.165108,2.526069,0.000000,1.042644,0.0,5.092123,2.636915,5.241840,0.137504,0.000000,0.000000,0.863938,0.014355,0.0
SIDM00884,0.250962,3.234195,0.378512,0.238787,4.765535,0.028569,0.000000,0.111031,0.014355,0.084064,1.028569,3.192194,0.070389,8.261578,0.0,0.0,0.056584,0.000000,0.137504,0.000000,3.351911,0.731183,7.599392,0.028569,0.150560,0.000000,0.097611,3.878725,0.0,6.747924,1.028569,0.298658,2.097611,0.214125,0.000000,0.056584,6.006747,3.960697,0.400538,0.286881,0.0,4.984589,1.608809,3.481557,0.000000,0.000000,0.084064,1.056584,0.000000,0.0
SIDM00877,0.432959,4.364572,0.000000,0.042644,3.621759,0.555816,0.275007,0.632268,0.014355,1.163499,1.485427,3.166715,0.111031,5.751142,0.0,0.0,2.575312,0.056584,0.790772,0.014355,5.272023,0.000000,4.338424,0.000000,0.594549,0.389567,2.204767,6.407353,0.0,5.269407,2.056584,2.347666,0.739848,0.201634,0.226509,2.247928,3.806324,3.109361,1.130931,0.286881,0.0,4.411426,2.347666,4.349082,0.226509,0.014355,0.070389,0.516015,0.014355,0.0


In [ ]:
mrmr_scores = fs.naive_test_regression(mrmr_feature_selected_data, label_data, cv=5)

c:\Users\l8105\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\l8105\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
## Does normalisation help?

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score

scaler = Normalizer()
models = [LinearRegression(), Ridge(), Lasso(), ElasticNet(), LinearSVR(max_iter=100000), KNeighborsRegressor(), RandomForestRegressor(), MLPRegressor(hidden_layer_sizes=(128, 64, 32), max_iter=20000, alpha=1)]

for model in models:
    pipe = Pipeline([('scaler', scaler), ('regressor', model)])
    scores = cross_val_score(pipe, mrmr_feature_selected_data, label_data, cv=5, scoring='neg_mean_squared_error')
    print(f'------------ {model.__class__.__name__}')
    print(f'NORM mean: {-scores.mean():.4f}, std: {scores.std():.4f}')

    scores = cross_val_score(model, mrmr_feature_selected_data, label_data, cv=5, scoring='neg_mean_squared_error')
    print(f'NONE mean: {-scores.mean():.4f}, std: {scores.std():.4f}')




------------ LinearRegression
NORM mean: 1.9162, std: 1.1885
NONE mean: 1.8649, std: 1.0800
------------ Ridge
NORM mean: 0.5671, std: 0.1698
NONE mean: 0.3456, std: 0.1518
------------ Lasso
NORM mean: 1.0552, std: 0.2627
NONE mean: 1.0552, std: 0.2627
------------ ElasticNet
NORM mean: 1.0552, std: 0.2627
NONE mean: 0.9474, std: 0.2491
------------ LinearSVR
NORM mean: 0.6393, std: 0.2244
NONE mean: 0.8301, std: 0.3932
------------ KNeighborsRegressor
NORM mean: 0.6364, std: 0.2343
NONE mean: 0.6428, std: 0.2172
------------ RandomForestRegressor
NORM mean: 0.4921, std: 0.1574
NONE mean: 0.5247, std: 0.1058
------------ MLPRegressor
NORM mean: 0.4669, std: 0.5075
NONE mean: 0.2959, std: 0.0891


In [ ]:
## Feature Selection with GFFS 

from dynmarker.FeatureSelection import greedy_forward_select_evaluate
from sklearn.model_selection import KFold

mrmr_scores = list(zip(successive_scores, mrmr_feature_selected))
outer_cv = KFold(n_splits=5, shuffle=True)
inner_cv = KFold(n_splits=5, shuffle=True)

gffs_feature_selected, final_scores = greedy_forward_select_evaluate(feature_data, label_data, K=20, model=KNeighborsRegressor(), ranked_features=mrmr_scores, scoring_method='neg_mean_squared_error',outer_cv=outer_cv, inner_cv=inner_cv, verbose=1) 

------------ Outer CV: 1
Feature Selected: 5033, Score: -0.5987792584321773, Feature Size: 2
Feature Selected: 8182, Score: -0.443185124156089, Feature Size: 3
Feature Selected: 12541, Score: -0.4115585563988402, Feature Size: 4
Feature Selected: 18835, Score: -0.440999792350728, Feature Size: 5
Feature Selected: 5642, Score: -0.48105902252864235, Feature Size: 6
Feature Selected: 8108, Score: -0.44400933895872574, Feature Size: 7
Feature Selected: 11308, Score: -0.39472315449514284, Feature Size: 8
Feature Selected: 19146, Score: -0.4306259343529269, Feature Size: 9
Feature Selected: 4817, Score: -0.4463934815565681, Feature Size: 10
Feature Selected: 16792, Score: -0.4037860811144013, Feature Size: 11
Feature Selected: 15615, Score: -0.4114486624867201, Feature Size: 12
Feature Selected: 13848, Score: -0.4061925723093463, Feature Size: 13
Feature Selected: 18564, Score: -0.417532421832688, Feature Size: 14
Feature Selected: 10752, Score: -0.4195528996023613, Feature Size: 15
Feature 

## Run 2 - Feature Selection Using Pipeline

In [10]:
## LOADING DATA

import pickle
import dynmarker.FeatureSelection as fs 

# Load ribociclib data from data/preprocessed 

with open('data/preprocessed/ribociclib_data.pkl', 'rb') as f:
    ribociclib_data = pickle.load(f)

feature_data, label_data = fs.create_feature_and_label(ribociclib_data)


In [16]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.feature_selection import f_regression, SelectKBest
# from sklearn import set_config
# set_config(transform_output = "pandas")

import pandas as pd 

from functools import partial

scaler = Normalizer()
feature_selector = SelectKBest(score_func=f_regression, k=20)
model = ElasticNet()

scores, pval = f_regression(feature_data, label_data)
scores # score for each feature at their respective index

K = 20
mrmr_score_func = partial(fs.mrmr_select_fcq_sklearn, K=K)
mrmr_selector = SelectKBest(mrmr_score_func, k=K)


# scaler.fit(feature_data)

# feature_data = scaler.transform(feature_data)
# feature_data

pipe = Pipeline([('feature_selector', mrmr_selector), ('regressor', model)])

scores = cross_val_score(pipe, feature_data, label_data, cv=5, scoring='neg_mean_squared_error')

print(f'NORM mean: {-scores.mean():.4f}, std: {scores.std():.4f}')

NORM mean: 1.1379, std: 0.3477


In [15]:
pipe = Pipeline([('feature_selector', feature_selector), ('regressor', model)])

scores = cross_val_score(pipe, feature_data, label_data, cv=5, scoring='neg_mean_squared_error')

print(f'NORM mean: {-scores.mean():.4f}, std: {scores.std():.4f}')

NORM mean: 1.1303, std: 0.3503


## Run 3 - Random Feature Selection

Without using sci-kit learn's pipeline for now, given compatability issues with pandas dataframe and column name tracebacks. 

In [2]:
## LOADING DATA

import pickle
import dynmarker.FeatureSelection as fs 

# Load ribociclib data from data/preprocessed 

with open('data/preprocessed/ribociclib_data.pkl', 'rb') as f:
    ribociclib_data = pickle.load(f)

feature_data, label_data = fs.create_feature_and_label(ribociclib_data)

In [3]:
y_std = label_data.std()
y_std

1.0169617839067444

In [4]:
import pandas as pd
import numpy as np 

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, ElasticNet
from sklearn.svm import LinearSVR, LinearSVC
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score

In [5]:
# create a Kfold object and test for cross-validation

outer_cv = KFold(n_splits=5, shuffle=True)

feature_selected_per_fold = []
for i, (train_index, test_index) in enumerate(outer_cv.split(feature_data, label_data)):
    X_train, X_test = feature_data.iloc[train_index,:], feature_data.iloc[test_index,:]
    y_train, y_test = label_data.iloc[train_index], label_data.iloc[test_index]
    print(f'------------ Outer CV: {i+1}')
    # print shape of train and test data
    print(f'X_train: {X_train.shape}, y_train: {y_train.shape}')
    print(f'X_test: {X_test.shape}, y_test: {y_test.shape}')

------------ Outer CV: 1
X_train: (35, 19221), y_train: (35,)
X_test: (9, 19221), y_test: (9,)
------------ Outer CV: 2
X_train: (35, 19221), y_train: (35,)
X_test: (9, 19221), y_test: (9,)
------------ Outer CV: 3
X_train: (35, 19221), y_train: (35,)
X_test: (9, 19221), y_test: (9,)
------------ Outer CV: 4
X_train: (35, 19221), y_train: (35,)
X_test: (9, 19221), y_test: (9,)
------------ Outer CV: 5
X_train: (36, 19221), y_train: (36,)
X_test: (8, 19221), y_test: (8,)


In [6]:
# conducting grand random feature selection

outer_cv = KFold(n_splits=3, shuffle=True)
k = 100
# model = MLPRegressor(hidden_layer_sizes=(128, 64, 32), max_iter=20000, alpha=1)
model = LinearRegression()

print(f'------------ {outer_cv.get_n_splits()}-Fold Cross-validation on feature selection ------------')

feature_selected_per_fold = []
scores = []
for i, (train_index, test_index) in enumerate(outer_cv.split(feature_data, label_data)):
    X_train, X_test = feature_data.iloc[train_index,:], feature_data.iloc[test_index,:]
    y_train, y_test = label_data.iloc[train_index], label_data.iloc[test_index]
    print(f'------------ Outer CV: {i+1}')
    
    random_feature_train, random_indices = fs.grand_random_selection(X_train, k=k)

    feature_selected_per_fold.append(random_indices)

    # fit a model on the selected features
    model = model 
    model.fit(random_feature_train, y_train)

    # evaluate the model on the test set
    random_feature_test = X_test.iloc[:, random_indices]
    y_pred = model.predict(random_feature_test)
    mse = mean_squared_error(y_test, y_pred)
    scores.append(mse)
    print(f'MSE: {mse:.4f}')
    print(f'selected features: {random_indices}')

# print model name and selected K value
print(f'--------------- RANDOM FEATURE SELECTION \n--------------- {k} features with {model.__class__.__name__}')
# print the average score and std
print(f'Average MSE: {np.mean(scores):.4f}, std: {np.std(scores):.4f}')

# find the features that are selected in all folds
common_features = set(feature_selected_per_fold[0])
for i in range(1, len(feature_selected_per_fold)):
    common_features = common_features.intersection(set(feature_selected_per_fold[i]))

print(f'Number of Common Features: {len(common_features)}, Feature Indices: {common_features}')


------------ 3-Fold Cross-validation on feature selection ------------
------------ Outer CV: 1
MSE: 1.4195
selected features: [  573   883  5634  8038 11942  3785  6369  6506 18202 13860  7935 14239
 10793  8408  4557 17070 18611 11837 17998 14953  1336 12151 18201 16595
  4332 14687  4447  2595  1166 12478 13633 10905  2145 13875 13267  1990
  2115  1236 16917 17458 16585  2289 11292 18513  2285 17012  3939  7008
 10626  9463  9008  5942 13719  9607 11788 17248 11394  4123  2895  6821
  2645 16325  6010  8595  5590  8657  6789 17555  7005  5879  2562 12164
 12875  7305  6161   102  3475  8474 17501  8919 13143 15671  3627  5593
 18681   747 14489  3870 13368 11188 17855 11592 12916  9760 18564 13618
 16831 13648  8514  8840]
------------ Outer CV: 2
MSE: 0.9037
selected features: [ 2384  3367  9030  6461 10467 12097  7748  7189 15898  6630 18331 11166
  9594 10583 16789 10186 14554 11021  8344  7388  2544 13304 14766  6531
  3090 18129 15992  2533 14363  1210 17725  9695  3011  6107 

In [7]:
from sklearn.base import BaseEstimator

class EvaluationInstance:

    def __init__(self, model: BaseEstimator, k: int, selected_feature_indices):
        self.model = model
        self.k = k
        self.selected_feature_indices = selected_feature_indices

        # training data and test data indices

        self.train_indices = None
        self.test_indices = None
    
    def fit(self, X: pd.DataFrame, y, train_indices):
        
        self.train_indices = train_indices

        # select the features
        selected_feature_train = X.iloc[train_indices, self.selected_feature_indices]

        # fit the model
        self.model.fit(selected_feature_train, y.iloc[train_indices])
    
    def predict(self, X: pd.DataFrame, y, test_indices):

        # check if the model is fitted
        if self.train_indices is None:
            raise Exception('The model is not fitted yet. Please fit the model first.')

        self.test_indices = test_indices

        # select the features
        selected_feature_test = X.iloc[test_indices, self.selected_feature_indices]

        # predict
        y_pred = self.model.predict(selected_feature_test)

        # return the predicted values and the actual values
        return (y_pred, y.iloc[test_indices])
    
    def get_model(self):
        return self.model
    
    def get_model_name(self):
        return self.model.__class__.__name__


In [10]:
from sklearn.base import clone 
from joblib import Parallel, delayed, cpu_count

regression_models = [LinearRegression(), 
                     Lasso(), 
                     ElasticNet(), 
                     LinearSVR(max_iter=100000), 
                     KNeighborsRegressor(), 
                     RandomForestRegressor(), 
                     MLPRegressor(hidden_layer_sizes=(128, 64, 32), max_iter=20000, alpha=1)]

# regression_models = [Lasso(), ElasticNet(), KNeighborsRegressor()]
k_ranges = []
# generate exponential growth from 1, such that 1,2,4,8...

for i in range(15):
    k_ranges.append(2**i)

# print(k_range)
iterations = 10
fold_split = 5
evaluation_df = pd.DataFrame(columns=['i', 'model_name', 'k', 'model', 'cv_number', 'feature_indices', 'eval_score'])


def parallel_func(i, k, model):
    outer_cv = KFold(n_splits=fold_split, shuffle=True)
    scores = []
    for j, (train_index, test_index) in enumerate(outer_cv.split(feature_data, label_data)):
        X_train, X_test = feature_data.iloc[train_index,:], feature_data.iloc[test_index,:]
        y_train, y_test = label_data.iloc[train_index], label_data.iloc[test_index]
        # print(f'------------ CV Numver: {i}')   
        
        random_feature_train, random_indices = fs.grand_random_selection(feature_data.iloc[train_index,:], k=k)

        # print(f'{i}, {j}, {k}, {model.__class__.__name__}, {random_indices}')

        model = clone(model)

        model.fit(X_train.iloc[:, random_indices], y_train)

        y_pred = model.predict(X_test.iloc[:, random_indices])
        mse = mean_squared_error(y_test, y_pred)
        scores.append(mse)
        # print(f'MSE: {mse:.4f}')

        new_df = pd.DataFrame({'i': [i], 'model_name': [model.__class__.__name__], 'k': [k], 'model': [model], 'cv_number': [j], 'feature_indices': [random_indices], 'eval_score': [mse]})

        # add the evaluation instance to the dataframe
        # evaluation_df = pd.concat((evaluation_df, new_df), ignore_index=True)
    
    return new_df

def single_loop_func():
    total_count = len(regression_models) * len(k_ranges) * iterations * fold_split
    print(f'------------ Total Count {total_count} ------------')
    evaluation_df = pd.DataFrame(columns=['i', 'model_name', 'k', 'model', 'cv_number', 'feature_indices', 'eval_score'])
    for model in regression_models:
        for k in k_ranges:
            # print(f'------------ {outer_cv.get_n_splits()}-Fold on {k} features with {model.__class__.__name__} ------------')
            for i in range(iterations):
                outer_cv = KFold(n_splits=fold_split, shuffle=True)
                scores = []
                for j, (train_index, test_index) in enumerate(outer_cv.split(feature_data, label_data)):
                    X_train, X_test = feature_data.iloc[train_index,:], feature_data.iloc[test_index,:]
                    y_train, y_test = label_data.iloc[train_index], label_data.iloc[test_index]
                    # print(f'------------ CV Numver: {i}')   
                    
                    random_feature_train, random_indices = fs.grand_random_selection(feature_data.iloc[train_index,:], k=k)

                    # print(f'{i}, {j}, {k}, {model.__class__.__name__}, {random_indices}')

                    # print(f'{i}, {j}, {k}, {model.__class__.__name__}, {random_indices}')

                    model = clone(model)

                    model.fit(X_train.iloc[:, random_indices], y_train)

                    y_pred = model.predict(X_test.iloc[:, random_indices])
                    mse = mean_squared_error(y_test, y_pred)
                    scores.append(mse)
                    # print(f'MSE: {mse:.4f}')

                    new_df = pd.DataFrame({'i': [i], 'model_name': [model.__class__.__name__], 'k': [k], 'model': [model], 'cv_number': [j], 'feature_indices': [random_indices], 'eval_score': [mse]})

                    # modelEval = EvaluationInstance(clone(model), k, random_indices)

                    # # fit a model on the selected features
                    # modelEval.fit(feature_data, label_data, train_index)

                    # # evaluate the model on the test set
                    # y_pred, y_test = modelEval.predict(feature_data, label_data, test_index)

                    # mse = mean_squared_error(y_test, y_pred)
                    # scores.append(mse)
                    # # print(f'MSE: {mse:.4f}')

                    # new_df = pd.DataFrame({'i': [i], 'model_name': [modelEval.get_model_name()], 'k': [k], 'model': [modelEval], 'cv_number': [i], 'feature_indices': [random_indices], 'eval_score': [mse]})

                    # add the evaluation instance to the dataframe
                    evaluation_df = pd.concat((evaluation_df, new_df), ignore_index=True)

                # print model name and selected K value
                print(f'Count {evaluation_df.shape[0]}/{total_count}, {i}-th iter: {k} features with {model.__class__.__name__}, Average MSE: {np.mean(scores):.4f}, std: {np.std(scores):.4f}')
                # # print the average score and std
                # print(f'Average MSE: {np.mean(scores):.4f}, std: {np.std(scores):.4f}')
    return evaluation_df


#### UNCOMMENT CODE BELOW TO RUN THE CODE AS SINGLE CORE ####
#### THIS ALLOWS FOR MESSAGES TO BE PRINTED ####

evaluation_df = single_loop_func()

#### UNCOMMENT CODE BELOW TO RUN THE CODE AS MULTI-CORE ####
#### THIS ALLOWS FOR MESSAGES TO BE PRINTED ####

# all_cpus = cpu_count()
# half_cpus = int(all_cpus/2)

# eval_list_total = []
# for model in regression_models:
#     evaluation_list = Parallel(n_jobs=half_cpus)(delayed(parallel_func)(i, k, model) for i in range(iterations) for k in k_ranges)
#     for evaluation in evaluation_list:
#         eval_list_total.append(evaluation)
#     print(f'--- Finished {model.__class__.__name__} using {half_cpus} cores ---')


# # concat the list of dataframes into one dataframe
# evaluation_df = pd.concat(eval_list_total, ignore_index=True)



------------ Total Count 5250 ------------
Count 5/5250, 0-th iter: 1 features with LinearRegression, Average MSE: 1.1011, std: 0.2357
Count 10/5250, 1-th iter: 1 features with LinearRegression, Average MSE: 1.2210, std: 0.2930
Count 15/5250, 2-th iter: 1 features with LinearRegression, Average MSE: 0.9812, std: 0.3079
Count 20/5250, 3-th iter: 1 features with LinearRegression, Average MSE: 1.0099, std: 0.5454
Count 25/5250, 4-th iter: 1 features with LinearRegression, Average MSE: 1.0449, std: 0.5498
Count 30/5250, 5-th iter: 1 features with LinearRegression, Average MSE: 1.1098, std: 0.7371
Count 35/5250, 6-th iter: 1 features with LinearRegression, Average MSE: 1.0915, std: 0.4708
Count 40/5250, 7-th iter: 1 features with LinearRegression, Average MSE: 1.0958, std: 0.3454
Count 45/5250, 8-th iter: 1 features with LinearRegression, Average MSE: 1.0549, std: 0.2832
Count 50/5250, 9-th iter: 1 features with LinearRegression, Average MSE: 1.1017, std: 0.6264
Count 55/5250, 0-th iter: 2 

In [11]:
evaluation_df.head()

,i,model_name,k,model,cv_number,feature_indices,eval_score
0,0,LinearRegression,1,LinearRegression(),0,[1462],1.398870
1,0,LinearRegression,1,LinearRegression(),1,[6969],0.745963
2,0,LinearRegression,1,LinearRegression(),2,[5420],0.928662
3,0,LinearRegression,1,LinearRegression(),3,[3255],1.156988
4,0,LinearRegression,1,LinearRegression(),4,[16622],1.274824


In [12]:
evaluation_df.tail()

,i,model_name,k,model,cv_number,feature_indices,eval_score
5245,9,MLPRegressor,16384,"MLPRegressor(alpha=1, hidden_layer_sizes=(128,...",0,"[19035, 11749, 16828, 1157, 4269, 8180, 12964,...",5.171823
5246,9,MLPRegressor,16384,"MLPRegressor(alpha=1, hidden_layer_sizes=(128,...",1,"[4166, 10742, 11985, 14185, 5172, 18808, 6122,...",4.090701
5247,9,MLPRegressor,16384,"MLPRegressor(alpha=1, hidden_layer_sizes=(128,...",2,"[12425, 6702, 9181, 7612, 16921, 18611, 900, 1...",1.110700
5248,9,MLPRegressor,16384,"MLPRegressor(alpha=1, hidden_layer_sizes=(128,...",3,"[12540, 841, 8585, 12920, 10174, 4206, 5161, 2...",1.663837
5249,9,MLPRegressor,16384,"MLPRegressor(alpha=1, hidden_layer_sizes=(128,...",4,"[16212, 6793, 12390, 9803, 754, 3334, 8423, 13...",1.704813


In [15]:
# given the model instances are too large to store in the dataframe, we only store the model name and the feature indices

evaluation_df_lite = evaluation_df[['i', 'model_name', 'k', 'cv_number', 'feature_indices', 'eval_score']]
evaluation_df_lite.head()

,i,model_name,k,cv_number,feature_indices,eval_score
0,0,LinearRegression,1,0,[1462],1.398870
1,0,LinearRegression,1,1,[6969],0.745963
2,0,LinearRegression,1,2,[5420],0.928662
3,0,LinearRegression,1,3,[3255],1.156988
4,0,LinearRegression,1,4,[16622],1.274824


In [16]:
evaluation_df_lite.to_pickle('data/results/ribociclib_ccle_gdsc2_evaluation_df_lite_random_feature_selection.pkl')

In [2]:
# load the dataframe from the pickle file

import pandas as pd

evaluation_df_lite = pd.read_pickle('data/results/ribociclib_ccle_gdsc2_evaluation_df_lite_random_feature_selection.pkl')

In [10]:
# plot a line plot the average score and std for each k value over the i iterations, show error bar as std

import matplotlib.pyplot as plt
import seaborn as sns

y_std = 1
# get rid of the outliers, 20 times the value of the label data's standard deviation is the cut-off
processed_evaluation_df = evaluation_df_lite[evaluation_df_lite['eval_score'] < y_std*20]
# processed_evaluation_df = evaluation_df_lite

# plot each model on the different figure, multi-figure plot

g = sns.FacetGrid(processed_evaluation_df, col='model_name', col_wrap=3, hue='model_name', height=4, aspect=1.5, sharey=False)
g.map(sns.lineplot, 'k', 'eval_score')
g.add_legend()
# plt.grid()
plt.show()